In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models

### Specifying the training and testing paths

In [12]:
# Paths
train_dir = 'dataset/training'
test_dir = 'dataset/testing'

### Loading the data using ImageDataGenerator

In [13]:
# Load data using ImageDataGenerator
img_height = 224
img_width = 224
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=False
)

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


### Identifying classes from dataset

In [14]:
# Class names
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


### Normalization

In [15]:
# Normalize
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

### AUTOTUNE

In [16]:
# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### CNN model architecture

In [17]:
# Build the CNN model
model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,168,900 (42.61 MB)

 Trainable params: 11,168,900 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
epochs_num = 20

history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs_num
)

# Save model
model.save(f'tensorflow_brain_tumor_model_{epochs_num}_epoch.h5')

Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 158s 859ms/step - accuracy: 0.6112 - loss: 0.9302 - val_accuracy: 0.8085 - val_loss: 0.4806
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 151s 846ms/step - accuracy: 0.8717 - loss: 0.3586 - val_accuracy: 0.8665 - val_loss: 0.3171
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 153s 856ms/step - accuracy: 0.9147 - loss: 0.2259 - val_accuracy: 0.9146 - val_loss: 0.2351
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 153s 854ms/step - accuracy: 0.9553 - loss: 0.1430 - val_accuracy: 0.9245 - val_loss: 0.1992
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 150s 839ms/step - accuracy: 0.9631 - loss: 0.1070 - val_accuracy: 0.9588 - val_loss: 0.1240
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 152s 849ms/step - accuracy: 0.9833 - loss: 0.0517 - val_accuracy: 0.9359 - val_loss: 0.2149
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 153s 857ms/step - accuracy: 0.9818 - loss: 0.0508 - val_accuracy: 0.9474 - val_loss: 0.1983
Epoch 8/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 151s 845ms/step - accuracy: 0.9793 -

### Printing the history of the training process

In [19]:
history.history

{'accuracy': [0.726190447807312,
  0.8778011202812195,
  0.9173669219017029,
  0.94992995262146,
  0.96253502368927,
  0.9819678068161011,
  0.981792688369751,
  0.9833683371543884,
  0.98757004737854,
  0.9901960492134094,
  0.9912465214729309,
  0.993172287940979,
  0.9957982897758484,
  0.9942227005958557,
  0.994397759437561,
  0.9922969341278076,
  0.9926470518112183,
  0.9954481720924377,
  0.9889705777168274,
  0.9970238208770752],
 'loss': [0.6743874549865723,
  0.33378106355667114,
  0.22030577063560486,
  0.1503690928220749,
  0.10202695429325104,
  0.05597623810172081,
  0.04974419251084328,
  0.04763321205973625,
  0.036144960671663284,
  0.028012091293931007,
  0.022852938622236252,
  0.01914541982114315,
  0.015429514460265636,
  0.021705059334635735,
  0.016616104170680046,
  0.02235962636768818,
  0.02187303826212883,
  0.013939103111624718,
  0.03876606374979019,
  0.00872111227363348],
 'val_accuracy': [0.8085430860519409,
  0.8665140867233276,
  0.9145690202713013,
 

### Converting the history in a DataFrame object

In [20]:
epoch_numbers = np.arange(1, 21)

epoch_numbers

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [21]:
history_df = pd.DataFrame(history.history)

history_df.insert(0, 'epoch', epoch_numbers)

history_df

,epoch,accuracy,loss,val_accuracy,val_loss
0,1,0.726190,0.674387,0.808543,0.480638
1,2,0.877801,0.333781,0.866514,0.317135
2,3,0.917367,0.220306,0.914569,0.235129
3,4,0.949930,0.150369,0.924485,0.199164
4,5,0.962535,0.102027,0.958810,0.124000
5,6,0.981968,0.055976,0.935927,0.214858
6,7,0.981793,0.049744,0.947368,0.198294
7,8,0.983368,0.047633,0.967201,0.123687
8,9,0.987570,0.036145,0.962624,0.153690
9,10,0.990196,0.028012,0.967201,0.143604


### Converting the percentage

In [22]:
history_df['accuracy'] = history_df['accuracy'] * 100
history_df['loss'] = history_df['loss'] * 100

history_df['val_accuracy'] = history_df['val_accuracy'] * 100
history_df['val_loss'] = history_df['val_loss'] * 100

history_df

,epoch,accuracy,loss,val_accuracy,val_loss
0,1,72.619045,67.438745,80.854309,48.063752
1,2,87.780112,33.378106,86.651409,31.713495
2,3,91.736692,22.030577,91.456902,23.512863
3,4,94.992995,15.036909,92.448515,19.916449
4,5,96.253502,10.202695,95.881009,12.399957
5,6,98.196781,5.597624,93.592680,21.485846
6,7,98.179269,4.974419,94.736844,19.829395
7,8,98.336834,4.763321,96.720064,12.368729
8,9,98.757005,3.614496,96.262395,15.368970
9,10,99.019605,2.801209,96.720064,14.360406


### Saving the results in a CVS file

In [23]:
history_df.to_csv(f'results/tensorflow_training_{epochs_num}_epochs.csv', index = False)